In [0]:
from pyspark.sql.functions import sum, countDistinct, broadcast, col
spark.conf.set("spark.sql.adaptive.enabled", "false")

In [0]:

zipcodes_df = spark.read.csv("/Volumes/mp_spark_deep_ddve/default/zipcode_data/zipcode_data.csv", header=True, inferSchema=True)
customers_df = spark.read.table("hive_metastore.default.fake_customers")

In [0]:
# zipcodes_df.show(100)

agg_zipcodes_df = (zipcodes_df
                    .withColumn("zipCode", col("zipCode").cast("string"))
                    .groupBy("zipCode")
                    .agg(sum("population").alias("total_population"))
)

# display(agg_zipcodes_df)

In [0]:
joined_df = (customers_df
            .join(broadcast(agg_zipcodes_df), customers_df.zip_code == agg_zipcodes_df.zipCode, how="inner")
            .groupby("zip_code")
            .agg(countDistinct("customer_id").alias("total_customers"))
)
        
joined_df.explain()

joined_df.write.format("delta").mode("overwrite").saveAsTable("hive_metastore.default.fake_broadcast_join_example")

== Physical Plan ==
*(5) HashAggregate(keys=[zip_code#1529], functions=[finalmerge_count(distinct merge count#2636L) AS count(customer_id#1522L)#2632L])
+- Exchange hashpartitioning(zip_code#1529, 200), ENSURE_REQUIREMENTS, [plan_id=2539]
   +- *(4) HashAggregate(keys=[zip_code#1529], functions=[partial_count(distinct customer_id#1522L) AS count#2636L])
      +- *(4) HashAggregate(keys=[zip_code#1529, customer_id#1522L], functions=[])
         +- Exchange hashpartitioning(zip_code#1529, customer_id#1522L, 200), ENSURE_REQUIREMENTS, [plan_id=2534]
            +- *(3) HashAggregate(keys=[zip_code#1529, customer_id#1522L], functions=[])
               +- *(3) Project [customer_id#1522L, zip_code#1529]
                  +- *(3) BroadcastHashJoin [zip_code#1529], [zipCode#1540], Inner, BuildRight, false, false
                     :- *(3) Project [customer_id#1522L, zip_code#1529]
                     :  +- *(3) Filter (if (isnotnull(_databricks_internal_edge_computed_column_skip_row#2648))